# Extracting Data
Investment Funds must be registered in Brazilian Securities and Exchange Comission (CVM). CVM has funds data on their daily returns, benchmark index, type of fund, issuer, manager and other informations related to it. All data is open to the public in CVM [website](https://dados.cvm.gov.br/group/fundos-de-investimento):
- Funds return: one database for daily, monthly, quarterly, anual
- Register info: funds name, manager, issuer, type, open/close
- Statement of Income: database with funds link to their state of income
- Investors profile: who owns funds quotas (other business, retirement funds, individual investors, professional investors)
- Performance metrics: how to calculate return, risk accordin to managers, collateral

For this study, I'm interested in the daily returns and register info. This way I can map funds by their features and follow their performance in time. The advantage to use daily data is that I can transform daily info into month, quarter and annual.

## Importing libraries

In [290]:
import pandas as pd
import numpy as np
import requests
import zipfile
import os

In [291]:
# Creating parameters to download data
## Date paramenters to match CVM files
years = ['2024','2023','2022','2021','2020']
legacy = ['2020']

months = range(1,13)
month_list = []

for i in months:
    if i<10: 
        i = str('0'+str(i))
    else:   
        i = str(i)
    month_list.append(i)

To collect the data, I'll request directly from CVM website the zipfiles containing the [daily returns of financial funds](https://dados.cvm.gov.br/dataset/fi-doc-inf_diario). The main issue here, is that we have two types of repositories: (1) daily data organized in monthly zip files from the current year to 3 years ago (Y-3), (2) yearly zip file for older data. So in my study 2024,2023,2022 and 2021 will have monthly zip files, while [2020](https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/) will have only one zip file with 12 csv files inside.

To solve it, I'll just need to create a loop to identify which year is considered old/legacy by CVM. Previously I created a legacy list with 2020 so I can use it in the loop now.

URL with daily return data
URL model: dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/**DADOS/inf_diario_fi_202307**.zip
- replace the date on the URL '202307' with the loop

URL for legacy data
URL legacy model: dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/**DADOS/HIST/inf_diario_fi_2000**.zip
- replace only the year 2000

In [292]:
# Create empty dataframes to store return data
cvm_daily_return = pd.DataFrame()
cvm_legacy_return = pd.DataFrame()

# Create loop to download data from 2020 to July 2024
## I'll collect data for each year in our years list
for yyyy in years:
    try:
        if yyyy in legacy:  # Y-3 data is considered history and moved to a different directory, I'll call it legacy
            daily_return_url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/inf_diario_fi_{yyyy}.zip'
            download_url = requests.get(daily_return_url)
            zip_filename = f'inf_diario_fi_{yyyy}.zip' 
            with open(zip_filename,'wb') as zip_ref:
                zip_ref.write(download_url.content)
            with zipfile.ZipFile(zip_filename, 'r') as cvm_zip:
                legacy_csv = [pd.read_csv(cvm_zip.open(f), sep=';') for f in cvm_zip.namelist()]
                cvm_legacy_return = pd.concat(legacy_csv)
            os.remove(zip_filename)
        else:
            for mm in month_list:   # Y-3< data is called by year and month. So we need to run all month_list elements
                daily_return_url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{yyyy}{mm}.zip'
                download_url = requests.get(daily_return_url) 
                zip_filename = f'inf_diario_fi_{yyyy}{mm}.zip' 
                with open(zip_filename, 'wb') as zip_ref:
                    zip_ref.write(download_url.content)
                with zipfile.ZipFile(zip_filename) as cvm_zip:
                    for file_name in cvm_zip.namelist():
                        if file_name.endswith('.csv'):
                            with cvm_zip.open(file_name) as cvm_csv:
                                cvm_daily_return_temp = pd.read_csv(cvm_csv, sep=';')
                                cvm_daily_return = pd.concat([cvm_daily_return, cvm_daily_return_temp])
                os.remove(zip_filename)
        cvm_daily_return = pd.concat([cvm_daily_return,cvm_legacy_return])
    except:
        pass    # Avoid stopping the process in case it looks for current year and months yet to come (YYYYMM+1)

# Delete unused variables to clean memory
del cvm_csv, cvm_zip, download_url, file_name, legacy, mm, yyyy

In [293]:
# Transform 'DT_COMPTC' in datetime variable
cvm_daily_return['DT_COMPTC'] = pd.to_datetime(cvm_daily_return['DT_COMPTC'] , format='ISO8601')
# I'll also keep only data I need
cvm_daily_return.drop(columns=['TP_FUNDO','CAPTC_DIA','RESG_DIA','VL_TOTAL'], inplace=True)
# cvm_daily_return.info()

In [294]:
# To calculate the funds return, we need the first and last day of the month
cvm_daily_return['year_month'] = cvm_daily_return['DT_COMPTC'].dt.to_period('M')

# Create separate dataframes to first and last day
first_day  = cvm_daily_return.groupby(['year_month', 'CNPJ_FUNDO']).min().reset_index()
last_day = cvm_daily_return.groupby(['year_month', 'CNPJ_FUNDO']).max().reset_index()

# Concat first and last day dataframes
cvm_return = pd.concat([first_day, last_day]).sort_values(by='DT_COMPTC').reset_index(drop=True)
# cvm_return.sort_values(['CNPJ_FUNDO','DT_COMPTC'])

# cvm_daily_return.groupby(['year_month', 'CNPJ_FUNDO']).min()
cvm_return['DT_COMPTC'].unique()

<DatetimeArray>
['2020-01-01 00:00:00', '2020-01-02 00:00:00', '2020-01-03 00:00:00',
 '2020-01-06 00:00:00', '2020-01-07 00:00:00', '2020-01-08 00:00:00',
 '2020-01-09 00:00:00', '2020-01-10 00:00:00', '2020-01-13 00:00:00',
 '2020-01-14 00:00:00',
 ...
 '2024-07-24 00:00:00', '2024-07-25 00:00:00', '2024-07-26 00:00:00',
 '2024-07-29 00:00:00', '2024-07-30 00:00:00', '2024-07-31 00:00:00',
 '2024-08-01 00:00:00', '2024-08-02 00:00:00', '2024-08-05 00:00:00',
 '2024-08-06 00:00:00']
Length: 1168, dtype: datetime64[ns]

Python show not all funds have the first register at the first day of the month. Is this possible? Yes.
- A fund only register the value of its assets once it's approved by CVM. Which means our starting date isn't the same for all funds, will depend on when their documents were processed.

Is this bad for this analysis? No.
- Since I'm calculating the return (assets final value - assets start value), the number between those day won't bias the data. I can expect a small variation in value with smaller the window between those.

In [295]:
# Calculate the return 
cvm_return.sort_values(by=['CNPJ_FUNDO','year_month','DT_COMPTC'], inplace=True, ascending=True)
cvm_return['quota_return'] = cvm_return.groupby(['CNPJ_FUNDO','year_month'])[['VL_QUOTA']].diff()
cvm_return['assets_return'] = cvm_return.groupby(['CNPJ_FUNDO','year_month'])[['VL_PATRIM_LIQ']].diff()

cvm_return = cvm_return.dropna()

### Register data
The next step is to identify the funds. Investors know it by their name, not their register number. I'll also need to check their status, to see if the fund still active. Here I face a similar situation from return data, [current](https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv) and [legacy](https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi_hist.zip) data.

In [296]:
# Open current register file
cvm_register = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', sep=';', encoding='latin-1')

# Open legacy register file
## Issue: legacy has several csvs in one zip file
register_url = 'https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi_hist.zip'
download_register = requests.get(register_url)
register_zip = 'cad_fi_hist.zip'
with open(register_zip,'wb') as reg_ref:
    reg_ref.write(download_register.content)
with zipfile.ZipFile(register_zip,'r') as register_zip:
    reg_csv_lag = [pd.read_csv(register_zip.open(g), sep=';', encoding='latin-1') for g in register_zip.namelist()]
    legacy_register = pd.concat(reg_csv_lag, axis=0, ignore_index=True)
os.remove('cad_fi_hist.zip')

C:\Users\anaak\AppData\Local\Temp\ipykernel_17812\3861353282.py:2: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  cvm_register = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', sep=';', encoding='latin-1')


In [297]:
# Check if headers from current and legacy register dataframes are the same
## Print the shape of each dataframe to see if they are the same
if cvm_register.shape[1] > legacy_register.shape[1]:    # Analyze the shape(0_row, 1_col). We are interested in cols so shape[1]
    cur_lag = cvm_register.shape[1] - legacy_register.shape[1]
    print('Current register dataframe has',cur_lag, 'more columns')
else:
    lag_cur = legacy_register.shape[1] - cvm_register.shape[1] 
    print('Legacy register dataframe has',lag_cur,'more columns')
print('CVM register shape:',cvm_register.shape) # General idea of shape for current register dataframe
print('Legacy register shape:',legacy_register.shape)# General idea of shape for legacy register dataframe
print('\n')

# Find which columns are convergent and divergent between them
reg_cur_col = cvm_register.columns
reg_lag_col = legacy_register.columns

# Create an object for columns in common and columns in legacy and not current
common_cols = reg_cur_col.intersection(reg_lag_col)
cur_not_lag = reg_cur_col.difference(reg_lag_col)
lag_not_cur = reg_lag_col.difference(reg_cur_col)

print(common_cols.nunique(),'columns in COMMON between current and legacy:')
print(common_cols)
print('\n')
print(cur_not_lag.nunique(),'columns in CURRENT register dataframe and not in LEGACY:')
print(cur_not_lag)
print('\n')
print(lag_not_cur.nunique(),'columns in LEGACY register dataframe and not in CURRENT:')
print(lag_not_cur)

Legacy register dataframe has 23 more columns
CVM register shape: (79654, 41)
Legacy register shape: (1837518, 64)


29 columns in COMMON between current and legacy:
Index(['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT', 'DT_INI_SIT',
       'DT_INI_EXERC', 'DT_FIM_EXERC', 'CLASSE', 'DT_INI_CLASSE',
       'RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO',
       'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_ADM', 'INF_TAXA_ADM', 'DIRETOR',
       'CNPJ_ADMIN', 'ADMIN', 'PF_PJ_GESTOR', 'CPF_CNPJ_GESTOR', 'GESTOR',
       'CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',
       'CNPJ_CONTROLADOR', 'CONTROLADOR'],
      dtype='object')


12 columns in CURRENT register dataframe and not in LEGACY:
Index(['CD_CVM', 'CLASSE_ANBIMA', 'DT_CANCEL', 'DT_CONST', 'DT_INI_ATIV',
       'DT_PATRIM_LIQ', 'ENTID_INVEST', 'INF_TAXA_PERFM', 'INVEST_CEMPR_EXTER',
       'TAXA_PERFM', 'TP_FUNDO', 'VL_PATRIM_LIQ'],
      dtype='object')


35 columns in LEGACY register dataframe and not in CURRE

#### Comparing dataframes
On this step I noticed a difference between columns within current and legacy register dataframe, and why is that?
- According to CVM notes, they changed the infos required from funds along the years. This can happen due a law change or CVM don't see the need to ask for that information (e.g: legacy  'DT_INI_TAXA_ADM' has the date for when administration fee was charged)
- Some headers changed their name. Due to operational reasons, CVM data managers change the name of the columns to fit their system (e.g: legacy 'VL_TAXA_PERFM' and current 'TAXA_PERFM'. Both them have the performance rate for managers based on funds gains)
- New columns addition. With the law change, CVM may ask for new information (e.g: current 'CLASSE_ANBIMA')

Based on those difference, I'll work with the columns they have in common. In total, there are **29 in common columns** I can work with, but not all data in needed.

**Columns in common between current and legacy register:**

*'CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT', 'DT_INI_SIT',
'DT_INI_EXERC', 'DT_FIM_EXERC', 'CLASSE', 'DT_INI_CLASSE',
'RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO',
'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_ADM', 'INF_TAXA_ADM', 'DIRETOR',
'CNPJ_ADMIN', 'ADMIN', 'PF_PJ_GESTOR', 'CPF_CNPJ_GESTOR', 'GESTOR',
'CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',
'CNPJ_CONTROLADOR', 'CONTROLADOR'*

**The data I'll need to identify investment funds are:**

*'CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT','CLASSE', 'DT_INI_CLASSE', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO','CPF_CNPJ_GESTOR', 'GESTOR','CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',*

Based on CVMs [dictionary](https://dados.cvm.gov.br/dados/FI/CAD/META/meta_cad_fi.txt), I'll use the following cols:
| Column | Description|
| --- | ---|
|CNPJ_FUNDO| Investment fund register code |
|DENOM_SOCIAL| Investment fund name |
|DT_REG| Register date|
|SIT| Situation (Active, Deactive)|
|CLASSE| Type of assets|
|DT_INI_CLASSE| Date from when assets were purchased|
|CONDOM| Open/Close fund|
|FUNDO_COTAS| If fund has quotas or not|
|FUNDO_EXCLUSIVO| Exclusive fund|
|CPF_CNPJ_GESTOR| Manager register code|
|GESTOR| Manager name|
|CNPJ_AUDITOR| Audit firm register code|
|AUDITOR| Audit firm name|
|CNPJ_CUSTODIANTE| Issuer register code|
|CUSTODIANTE| Issuer name|

These columns will give me an idea of each fund structue. Those columns will tell me what type of assets each fund is working with, when they started trading those, who is the manager choosing the assets and who is issuing the quotas. And why does it matter?

I can analyze funds performance over the years and identify if a manager has better results than the others. Funds features (such as open/closed or exclusive/not) may indicate better performing funds due to private information or access to better assets. Issuer can hold a specific type of asset or only issue for a certain type of investor, and therefore have a different performance. By keeping audit firm data, I can point who are the big firms working with investment funds or a specific type of asset.

In [298]:
# Merging current and legacy register dataframes based on the columns I selected
main_cols = ['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT','CLASSE', 'DT_INI_CLASSE', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO','CPF_CNPJ_GESTOR', 'GESTOR','CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE']
cvm_complete_reg = cvm_register[main_cols].copy()
pd.concat([cvm_register,legacy_register], join='inner')
cvm_complete_reg.reset_index(inplace=True)
cvm_complete_reg.drop('index',axis=1, inplace=True)
cvm_complete_reg.shape

(79654, 15)

In [299]:
# Check data type
print(cvm_complete_reg.info())
# Convert 'DT_REG' and 'DT_INI_CLASSE' in datetime variables
cvm_complete_reg['DT_REG'] = pd.to_datetime(cvm_complete_reg['DT_REG'], format='ISO8601')   # ISO8601 sets date format to Year/month/day
cvm_complete_reg['DT_INI_CLASSE'] = pd.to_datetime(cvm_complete_reg['DT_INI_CLASSE'], format='ISO8601')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79654 entries, 0 to 79653
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CNPJ_FUNDO        79654 non-null  object
 1   DENOM_SOCIAL      79654 non-null  object
 2   DT_REG            79654 non-null  object
 3   SIT               79654 non-null  object
 4   CLASSE            66348 non-null  object
 5   DT_INI_CLASSE     66348 non-null  object
 6   CONDOM            66000 non-null  object
 7   FUNDO_COTAS       66354 non-null  object
 8   FUNDO_EXCLUSIVO   55864 non-null  object
 9   CPF_CNPJ_GESTOR   52508 non-null  object
 10  GESTOR            52508 non-null  object
 11  CNPJ_AUDITOR      51776 non-null  object
 12  AUDITOR           51776 non-null  object
 13  CNPJ_CUSTODIANTE  50955 non-null  object
 14  CUSTODIANTE       50955 non-null  object
dtypes: object(15)
memory usage: 9.1+ MB
None


In [300]:
# Check final result
print(cvm_complete_reg.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79654 entries, 0 to 79653
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CNPJ_FUNDO        79654 non-null  object        
 1   DENOM_SOCIAL      79654 non-null  object        
 2   DT_REG            79654 non-null  datetime64[ns]
 3   SIT               79654 non-null  object        
 4   CLASSE            66348 non-null  object        
 5   DT_INI_CLASSE     66348 non-null  datetime64[ns]
 6   CONDOM            66000 non-null  object        
 7   FUNDO_COTAS       66354 non-null  object        
 8   FUNDO_EXCLUSIVO   55864 non-null  object        
 9   CPF_CNPJ_GESTOR   52508 non-null  object        
 10  GESTOR            52508 non-null  object        
 11  CNPJ_AUDITOR      51776 non-null  object        
 12  AUDITOR           51776 non-null  object        
 13  CNPJ_CUSTODIANTE  50955 non-null  object        
 14  CUSTODIANTE       5095

In [301]:
# Check to see if I got the current data right
cvm_complete_reg.isna().sum()/len(cvm_complete_reg)

CNPJ_FUNDO          0.000000
DENOM_SOCIAL        0.000000
DT_REG              0.000000
SIT                 0.000000
CLASSE              0.167047
DT_INI_CLASSE       0.167047
CONDOM              0.171416
FUNDO_COTAS         0.166972
FUNDO_EXCLUSIVO     0.298667
CPF_CNPJ_GESTOR     0.340799
GESTOR              0.340799
CNPJ_AUDITOR        0.349989
AUDITOR             0.349989
CNPJ_CUSTODIANTE    0.360296
CUSTODIANTE         0.360296
dtype: float64

In [302]:
# Analyze which datapoints are empty, I'll use as reference the issuer feature (CUSTODIANTE) since it has the biggest percentage of nulls.
empty_reg = cvm_complete_reg[cvm_complete_reg['CUSTODIANTE'].isna() == True].sort_values(by='DT_REG', ascending=True)
# See dates with the most null values
empty_reg['DT_REG'].value_counts()/len(empty_reg)

DT_REG
2003-04-30    0.311196
2005-03-21    0.004774
2005-03-16    0.004669
2005-03-18    0.004599
2005-04-15    0.003972
                ...   
1969-10-14    0.000035
1969-08-01    0.000035
1969-06-18    0.000035
1969-01-10    0.000035
1961-04-21    0.000035
Name: count, Length: 5280, dtype: float64

In [303]:
empty_reg[empty_reg['DT_REG'] >= pd.to_datetime('2003-04-30')].count()

CNPJ_FUNDO          26173
DENOM_SOCIAL        26173
DT_REG              26173
SIT                 26173
CLASSE              15417
DT_INI_CLASSE       15417
CONDOM              15231
FUNDO_COTAS         15420
FUNDO_EXCLUSIVO     13694
CPF_CNPJ_GESTOR      1570
GESTOR               1570
CNPJ_AUDITOR         1042
AUDITOR              1042
CNPJ_CUSTODIANTE        0
CUSTODIANTE             0
dtype: int64

My null values analysis show that I have aound 36% of missing data for my issuer features. Diving deep into why I detected it can happen if the fund ceases its activities ('CANCELADO') or is in the process of opening. This means that neither of those funds will provide me insight on my return analysis.

To clean my register dataset, I'll remove rows with empty issuer.

But won't that be a problem? You'll lose 36% of your data!
- No. Since I'm focused on funds return, if a fund is not operational (CENCELADA) or in process to begin its operations, I'm not expecting to have historical data on their performance (26k out of 28k null values). I acept to work with a missing value data of 2k.
 

In [304]:
cvm_complete_reg = cvm_complete_reg.dropna()
# Check for duplicates in register file
cvm_complete_reg.duplicated(['CNPJ_FUNDO']).value_counts()

### This means I have 1689 duplicated values in my register dataframe, why?

False    40439
True      1689
Name: count, dtype: int64

In [305]:
duplicated_register = cvm_complete_reg[cvm_complete_reg.duplicated(subset='CNPJ_FUNDO', keep=False)]
duplicated_register.sort_values(by=['CNPJ_FUNDO'], inplace=True)
duplicated_register.head(10)

C:\Users\anaak\AppData\Local\Temp\ipykernel_17812\3355904875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated_register.sort_values(by=['CNPJ_FUNDO'], inplace=True)


,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT,CLASSE,DT_INI_CLASSE,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE
5731,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2005-03-31,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5732,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2005-03-31,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5733,01.392.021/0001-62,BRADESCO H FC DE FUNDOS DE INVESTIMENTO RENDA ...,2005-04-07,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-31,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5734,01.392.021/0001-62,BRADESCO H FC DE FUNDOS DE INVESTIMENTO RENDA ...,2005-04-07,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-31,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5845,01.603.578/0001-03,ICATU VANGUARDA XI FUNDO DE INVESTIMENTO RENDA...,2005-03-23,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-11,Aberto,N,N,42.283.770/0001-39,ICATU SEGUROS S/A,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,60.746.948/0001-12,BANCO BRADESCO S.A.
5846,01.603.578/0001-03,ICATU VANGUARDA XI FUNDO DE INVESTIMENTO RENDA...,2005-03-23,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-11,Aberto,N,N,68.622.174/0001-20,ICATU VANGUARDA GESTÃO DE RECURSOS LTDA,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,60.746.948/0001-12,BANCO BRADESCO S.A.
5852,01.606.509/0001-45,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2005-03-31,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5853,01.606.509/0001-45,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2005-03-31,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
6038,01.873.685/0001-43,PREVI RENDA FIXA TÍTULOS PÚBLICOS HTM FUNDO DE...,2005-04-11,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-31,Aberto,N,S,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.
6039,01.873.685/0001-43,PREVI RENDA FIXA TÍTULOS PÚBLICOS HTM FUNDO DE...,2005-04-11,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-31,Aberto,N,S,33.754.482/0001-24,CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO...,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.


My duplicate analysis show an interesting behavior. Usually, analysist drop/delete duplicates in their datasets. The reason is becaus duplicates usually are viewed as an error. However, when we see what is happening in our register dataframe, I see that funds are registered two times (at least): first when the fund is opened to business and the second when the fund closes its activities.

Based on this behavior, I can't drop duplicates because in this case it is a feature rather than an error. Let's see if there are duplicated values even if the fund still in business.

In [306]:
fund_active = duplicated_register[~duplicated_register['SIT'].str.contains('CANCELADA')]
fund_active = fund_active[fund_active.duplicated(subset='CNPJ_FUNDO', keep=False)]
fund_active.head(10)

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT,CLASSE,DT_INI_CLASSE,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE
5731,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2005-03-31,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5732,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2005-03-31,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5733,01.392.021/0001-62,BRADESCO H FC DE FUNDOS DE INVESTIMENTO RENDA ...,2005-04-07,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-31,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5734,01.392.021/0001-62,BRADESCO H FC DE FUNDOS DE INVESTIMENTO RENDA ...,2005-04-07,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-31,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5845,01.603.578/0001-03,ICATU VANGUARDA XI FUNDO DE INVESTIMENTO RENDA...,2005-03-23,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-11,Aberto,N,N,42.283.770/0001-39,ICATU SEGUROS S/A,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,60.746.948/0001-12,BANCO BRADESCO S.A.
5846,01.603.578/0001-03,ICATU VANGUARDA XI FUNDO DE INVESTIMENTO RENDA...,2005-03-23,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-11,Aberto,N,N,68.622.174/0001-20,ICATU VANGUARDA GESTÃO DE RECURSOS LTDA,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,60.746.948/0001-12,BANCO BRADESCO S.A.
5852,01.606.509/0001-45,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2005-03-31,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5853,01.606.509/0001-45,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2005-03-31,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
6038,01.873.685/0001-43,PREVI RENDA FIXA TÍTULOS PÚBLICOS HTM FUNDO DE...,2005-04-11,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-31,Aberto,N,S,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.
6039,01.873.685/0001-43,PREVI RENDA FIXA TÍTULOS PÚBLICOS HTM FUNDO DE...,2005-04-11,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2005-03-31,Aberto,N,S,33.754.482/0001-24,CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO...,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.


This results show that a fund can be registered twice even if it still active. However, the condition of how the funding is running might be different. A fund can still be operational, but in a specific state, such as in procress of registering, on hold for analysis, or even starting the process of liquidation (selling assets to end the fund).

Based on these condition, I need to know what can I expect to see withing the status ('SIT') feature.

In [307]:
# Find unique labels for each situation that are not canceled (CANCELADA)
fund_active['SIT'].unique()
# Delete variables to clean memory space
del fund_active,duplicated_register

Status feature ('SIT') has 4 unique labels:
- 'EM ANÁLISE': CVM is analyzing the documents
- 'EM FUNCIONAMENTO NORMAL': operating business as usual
- 'FASE PRÉ-OPERACIONAL': 1 phase before opening to business
- 'LIQUIDAÇÃO': in process of selling assets to end the fund

These classes are important to tell us the story of each fund. So a fund can be registered twice even if it's not stopped its activities.

But status ('SIT') is not the only feature that can explain duplicated registers. Look for example what happend with fund 32.891.432/0001-26. All its features are the same, except for a change Manager ('GESTOR'), which means the fund at some point in time changed institutions managing their assets and had to issue a new registration.

### Merging datasets: daily returns + register data
Now it's time to merge our dataframes, return base and register data. Due to my duplicate analysis, I need to pay attention to how to merge data. For instance, a fund can have two different managers institution based on the date.

So my keys to merge the data will be the funds registration number ('CNPJ_FUNDO') and the register data ('DT_REG'). Naturally, another issue will appear: return data is in a daily frequency and register data is ponctual. This means I'll have a series of empty rows for the features coming from register dataframe. I'll fix it by coping each fund previous row, this means: since register occurs only when the fund is created or has some change, it's fair to use the data on the register/change day to further dates until the fund dissapears from our database (meaning they are ceased their operations).

In [308]:
# Merg key
## Substitute all dates before 2020 to '2020-01-01', this way I can merge datasets
df_reg = cvm_complete_reg
df_reg.sort_values(by='DT_REG', inplace=True, ascending=False)
df_reg.loc[df_reg['DT_REG'] < '2020-01-01','DT_REG'] = pd.to_datetime('2020-01-01')
# cvm_complete_reg = cvm_complete_reg.drop_duplicates(subset=['CNPJ_FUNDO','DT_REG'], keep='first')  # Drop older data
df_reg['year_month'] = df_reg['DT_REG'].dt.to_period('M')   # I'm gonna use year-month as a key

In [309]:
# Merge return and register dataframes based on register year-month and fund ID
fund_df = pd.merge(cvm_return, df_reg.drop(columns=['CLASSE', 'DT_INI_CLASSE']), how='left', on=['year_month','CNPJ_FUNDO'])
# Possible issue: fund register in a month and only starts to operate in the following month (on which my previous merge won't work)
fund_df.sort_values(by=['year_month','CNPJ_FUNDO'], inplace=True)

fund_df = fund_df.fillna(method='ffill')
fund_df


C:\Users\anaak\AppData\Local\Temp\ipykernel_17812\3333499208.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fund_df = fund_df.fillna(method='ffill')


,year_month,CNPJ_FUNDO,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST,quota_return,assets_return,DENOM_SOCIAL,DT_REG,SIT,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE
0,2020-01,00.017.024/0001-53,2020-01-31,27.256965,1.123787e+06,1,0.032469,3.386370e+03,FUNDO DE INVESTIMENTO RENDA FIXA EXPONENCIAL,2020-01-01,EM FUNCIONAMENTO NORMAL,Aberto,N,S,60.746.948/0001-12,BANCO BRADESCO S.A.,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
56,2020-01,00.068.305/0001-35,2020-01-31,27.181630,5.514803e+07,7740,0.068893,5.494508e+05,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2020-01-01,EM FUNCIONAMENTO NORMAL,Aberto,S,N,42.040.639/0001-40,CAIXA DISTRIBUIDORA DE TÍTULOS E VALORES MOBIL...,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,00.360.305/0001-04,CAIXA ECONOMICA FEDERAL
112,2020-01,00.071.477/0001-68,2020-01-31,9.990082,2.234333e+10,293889,0.010745,2.010797e+09,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...,2020-01-01,EM FUNCIONAMENTO NORMAL,Aberto,S,N,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.
168,2020-01,00.073.041/0001-08,2020-01-31,28.333920,2.062547e+07,984,0.083578,9.225857e+04,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...,2020-01-01,EM FUNCIONAMENTO NORMAL,Aberto,N,N,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,00.000.000/0001-91,BANCO DO BRASIL S.A.
224,2020-01,00.083.181/0001-67,2020-01-31,721.725453,1.651549e+10,4,30.869400,7.063950e+08,OPP I FUNDO DE INVESTIMENTO EM AÇÕES BDR NÍVEL...,2020-01-01,EM FUNCIONAMENTO NORMAL,Fechado,N,N,33.857.830/0001-99,OPPORTUNITY HDF ADMINISTRADORA DE RECURSOS LTDA.,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,42.272.526/0001-70,BNY MELLON BANCO S.A.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274767,2024-08,97.548.164/0001-90,2024-08-05,0.136757,1.754542e+07,1,0.000244,3.136832e+04,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...,2024-08-02,FASE PRÉ-OPERACIONAL,Aberto,N,N,06.195.084/0001-42,EMPIRICUS GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
1274823,2024-08,97.548.167/0001-23,2024-08-05,5.349470,2.712536e+09,10,0.004021,1.451171e+07,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...,2024-08-02,FASE PRÉ-OPERACIONAL,Aberto,N,N,06.195.084/0001-42,EMPIRICUS GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
1274879,2024-08,97.711.801/0001-05,2024-08-05,0.000000,0.000000e+00,0,0.000000,0.000000e+00,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...,2024-08-02,FASE PRÉ-OPERACIONAL,Aberto,N,N,06.195.084/0001-42,EMPIRICUS GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
1274938,2024-08,97.929.197/0001-80,2024-08-05,6.149075,1.330367e+08,12,0.086711,1.876009e+06,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...,2024-08-02,FASE PRÉ-OPERACIONAL,Aberto,N,N,06.195.084/0001-42,EMPIRICUS GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A


In [310]:
fund_df.isna().sum()

year_month          0
CNPJ_FUNDO          0
DT_COMPTC           0
VL_QUOTA            0
VL_PATRIM_LIQ       0
NR_COTST            0
quota_return        0
assets_return       0
DENOM_SOCIAL        0
DT_REG              0
SIT                 0
CONDOM              0
FUNDO_COTAS         0
FUNDO_EXCLUSIVO     0
CPF_CNPJ_GESTOR     0
GESTOR              0
CNPJ_AUDITOR        0
AUDITOR             0
CNPJ_CUSTODIANTE    0
CUSTODIANTE         0
dtype: int64

In [316]:
fund_list = fund_df[['CNPJ_FUNDO', 'DENOM_SOCIAL']].drop_duplicates()
reg_list = df_reg[['CNPJ_FUNDO', 'DENOM_SOCIAL']].drop_duplicates()
fund_list

,CNPJ_FUNDO,DENOM_SOCIAL
0,00.017.024/0001-53,FUNDO DE INVESTIMENTO RENDA FIXA EXPONENCIAL
56,00.068.305/0001-35,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...
112,00.071.477/0001-68,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...
168,00.073.041/0001-08,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...
224,00.083.181/0001-67,OPP I FUNDO DE INVESTIMENTO EM AÇÕES BDR NÍVEL...
...,...,...
1274767,97.548.164/0001-90,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...
1274823,97.548.167/0001-23,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...
1274879,97.711.801/0001-05,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...
1274938,97.929.197/0001-80,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...


In [317]:
reg_list = reg_list.rename(columns={'DENOM_SOCIAL':'DENOM_SOCIAL_REG'})
check = pd.merge(fund_list,reg_list, on='CNPJ_FUNDO', how='left')
check['correct_name'] = check['DENOM_SOCIAL'] == check['DENOM_SOCIAL_REG']
check['correct_name'].value_counts()

correct_name
False    1252625
True       21522
Name: count, dtype: int64

In [318]:
check[check['correct_name'] == False]

,CNPJ_FUNDO,DENOM_SOCIAL,DENOM_SOCIAL_REG,correct_name
27,00.754.543/0001-02,BANRISUL VIP FUNDO DE INVESTIMENTO RENDA FIXA ...,NaN,False
55,00.826.600/0001-03,ITAÚ BASIS OPEN RENDA FIXA REFERENCIADO DI FUN...,NaN,False
61,00.828.371/0001-66,FATOR MAX CORPORATIVO FUNDO DE INVESTIMENTO EM...,NaN,False
83,00.836.255/0001-99,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,NaN,False
105,00.902.484/0001-64,FAPA SENIOR FUNDO DE INVESTIMENTO MULTIMERCADO,NaN,False
...,...,...,...,...
1274142,97.548.164/0001-90,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...,DYNAMIC II - FUNDO DE INVESTIMENTO MULTIMERCAD...,False
1274143,97.548.167/0001-23,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...,JGP STRATEGY MASTER FUNDO DE INVESTIMENTO MULT...,False
1274144,97.711.801/0001-05,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...,953 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS D...,False
1274145,97.929.197/0001-80,EMPIRICUS DIGITAL CRYPTO FUNDO DE INVESTIMENTO...,LECT FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS ...,False


In [314]:
cvm_complete_reg[cvm_complete_reg['CNPJ_FUNDO'] == '00.754.543/0001-02']
# cvm_complete_reg[cvm_complete_reg['DENOM_SOCIAL'].str.contains('BANRISUL VIP FUNDO DE INVESTIMENTO RENDA FIXA')]

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT,CLASSE,DT_INI_CLASSE,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,year_month


In [315]:
fund_df[fund_df['CNPJ_FUNDO'] == '00.754.543/0001-02']

,year_month,CNPJ_FUNDO,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST,quota_return,assets_return,DENOM_SOCIAL,DT_REG,SIT,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE
1543,2020-01,00.754.543/0001-02,2020-01-31,5.235998,6546835.99,1,0.016377,20477.57,BANRISUL VIP FUNDO DE INVESTIMENTO RENDA FIXA ...,2020-01-01,EM FUNCIONAMENTO NORMAL,Aberto,N,N,93.026.847/0001-26,BANRISUL S.A. CORRETORA DE VALORES MOBILIÁRIOS...,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,92.702.067/0001-96,BANCO DO ESTADO DO RIO GRANDE DO SUL SA
1544,2020-02,00.754.543/0001-02,2020-02-28,5.248939,6563017.18,1,0.012171,15217.59,WHG SISTEMA II FUNDO DE INVESTIMENTO MULTIMERC...,2020-01-01,EM FUNCIONAMENTO NORMAL,Fechado,N,S,34.848.969/0001-39,WEALTH HIGH GOVERNANCE CAPITAL LTDA.,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,62.318.407/0001-19,S3 CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VA...
1545,2020-03,00.754.543/0001-02,2020-03-31,5.264034,6581891.22,1,0.014436,18049.47,G5 SCB II FUNDO DE INVESTIMENTO EM AÇÕES,2020-02-12,EM FUNCIONAMENTO NORMAL,Fechado,N,N,09.446.129/0001-00,G5 ADMINISTRADORA DE RECURSOS LTDA.,19.280.834/0001-26,NEXT AUDITORES INDEPENDENTES S/S LTDA.,62.285.390/0001-40,SINGULARE CORRETORA DE TITULOS E VALORES MOBIL...
1546,2020-04,00.754.543/0001-02,2020-04-30,5.276499,6597476.80,1,0.011852,14818.77,INTER ACCESS ATMOS FUNDO DE INVESTIMENTO EM CO...,2020-03-25,EM FUNCIONAMENTO NORMAL,Aberto,S,N,05.585.083/0001-41,INTER ASSET GESTÃO DE RECURSOS LTDA.,10.830.108/0001-65,GRANT THORNTON AUDITORES INDEPENDENTES LTDA.,18.945.670/0001-46,INTER DISTRIBUIDORA DE TÍTULOS E VALORES MOBIL...
1547,2020-05,00.754.543/0001-02,2020-05-29,5.286248,6609073.28,1,0.009134,15349.05,SKYLINE FUNDO DE INVESTIMENTO MULTIMERCADO CRÉ...,2020-04-08,EM FUNCIONAMENTO NORMAL,Fechado,N,N,11.392.069/0001-24,V8 CAPITAL GESTÃO DE INVESTIMENTOS LTDA.,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,62.232.889/0001-90,BANCO DAYCOVAL S.A.
1548,2020-06,00.754.543/0001-02,2020-06-30,5.295148,6604825.44,1,0.008418,10499.77,CANASTRA FUNDO DE INVESTIMENTO MULTIMERCADO CR...,2020-05-27,EM FUNCIONAMENTO NORMAL,Fechado,N,S,60.451.242/0001-23,BTG PACTUAL WM GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
1549,2020-07,00.754.543/0001-02,2020-07-31,5.302777,6614341.40,1,0.007302,9107.50,SICOOB LIQUIDEZ MASTER FUNDO DE INVESTIMENTO R...,2020-06-12,EM FUNCIONAMENTO NORMAL,Aberto,N,N,07.397.614/0001-06,BANCOOB DISTRIBUIDORA DE TÍTULOS E VALORES MOB...,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,02.038.232/0001-64,BANCO COOPERATIVO SICOOB S.A
1550,2020-08,00.754.543/0001-02,2020-08-31,5.308876,6621948.59,1,0.005772,7199.15,GVL CASH FUNDO DE INVESTIMENTO EM RENDA FIXA C...,2020-07-31,EM FUNCIONAMENTO NORMAL,Aberto,N,N,60.451.242/0001-23,BTG PACTUAL WM GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
1551,2020-09,00.754.543/0001-02,2020-09-30,5.311054,6624665.93,1,0.003264,4071.62,3KB FUNDO DE INVESTIMENTO DE AÇÕES INVESTIMENT...,2020-08-28,CANCELADA,Aberto,N,N,17.817.934/0001-13,4K INVESTIMENTOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
1552,2020-10,00.754.543/0001-02,2020-10-30,5.312301,6626220.77,1,0.014552,18151.48,SPARTA DINÂMICO L FEEDER FUNDO DE INVESTIMENTO...,2020-09-25,CANCELADA,Aberto,S,N,22.134.302/0001-95,LAKEWOOD GESTÃO DE RECURSOS LTDA.,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
